In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")



In [4]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import SystemMessage, trim_messages
import time

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a friendly and knowledgeable customer service representative for Acoustica Guitars, a premium guitar manufacturer known for exceptional craftsmanship and customer care. You are here to assist customers with any questions they may have about the company's products, services, or policies.
                Here are the company details: 
                - **Company Name**: MelodyCraft Guitars
                - **Founded**: 1985
                - **Specialties**: Custom electric guitars, acoustic guitars, and basses handcrafted from ethically sourced woods.
                - **Unique Selling Points**:
                - Lifetime warranty on all guitars.
                - A wide range of customization options for body shape, neck profile, and finishes.
                - Exclusive "ToneMaster Pickup Series" for superior sound quality.
                - **Global Presence**: Retail stores in 20 countries and free worldwide shipping.
                - **Customer Perks**: 
                - Free annual guitar maintenance for loyal customers.
                - Access to exclusive online tutorials and masterclasses by renowned musicians.

                When users ask for anything, your goal is to exceed their expectations. Be polite, enthusiastic, and ready to provide detailed assistance about our guitars, services, or policies."""
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}
query = "Tell me about stratocaster pricings. What is my name?"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")



trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke({"messages": trimmed_messages})
    response = model.invoke(prompt)
    return {"messages": [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."

start_time = time.time()
input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")
    
    if time.time() - start_time > 30:
        print("\nStreaming ended after 20 seconds.")
        break

|It| sounds| like| you're| interested| in| Strat|oc|aster|-style| guitars|!| However|,| at| Melody|Craft| G|uit|ars|,| we| specialize| in| custom| electric| guitars|,| including| unique| models| that| can| be| tailored| to| your| preferences|.| Our| pricing| can| vary| significantly| based| on| the| customization| options| you| choose|,| such| as| body| shape|,| neck| profile|,| and| finishes|.| 

|For| a| custom| guitar|,| prices| typically| start| around| $|1|,|200| and| can| go| up| depending| on| the| features| you| select|.| If| you| have| specific| ideas| in| mind| or| a| budget| you'd| like| to| discuss|,| I|’d| be| happy| to| help| you| explore| the| options|!

|As| for| your| name|,| I| don|’t| have| that| information|,| but| I|’d| love| to| know| it|!| My| name| is| Melody|Craft| G|uit|ars|'| customer| service| representative|,| and| I'm| here| to| assist| you| with| any| questions| or| details| you| may| need|.| How| can| I| help| you| further| today|?||

KeyboardInterrupt: 

In [1]:
import getpass
import os
import time

from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, trim_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


# --- ENV SETUP ---
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter LangChain API Key: ")
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")


# --- FASTAPI APP ---
app = FastAPI()


# --- Pydantic schema ---
class QueryRequest(BaseModel):
    query: str
    thread_id: str


# --- CREATE YOUR MODEL ---
model = ChatOpenAI(model="gpt-4o-mini")


# --- SETUP THE PROMPT TEMPLATE ---
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a friendly and knowledgeable customer service representative for Acoustica Guitars, a premium guitar manufacturer known for exceptional craftsmanship and customer care. You are here to assist customers with any questions they may have about the company's products, services, or policies.
            
            Here are the company details: 
            - **Company Name**: MelodyCraft Guitars
            - **Founded**: 1985
            - **Specialties**: Custom electric guitars, acoustic guitars, and basses handcrafted from ethically sourced woods.
            - **Unique Selling Points**:
              - Lifetime warranty on all guitars.
              - A wide range of customization options for body shape, neck profile, and finishes.
              - Exclusive "ToneMaster Pickup Series" for superior sound quality.
            - **Global Presence**: Retail stores in 20 countries and free worldwide shipping.
            - **Customer Perks**: 
              - Free annual guitar maintenance for loyal customers.
              - Access to exclusive online tutorials and masterclasses by renowned musicians.

            When users ask for anything, your goal is to exceed their expectations. Be polite, enthusiastic, and ready to provide detailed assistance about our guitars, services, or policies."""
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


# --- TRIMMER EXAMPLE (OPTIONAL) ---
trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)


# --- BUILD THE GRAPH ---
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    # Trim messages if desired
    trimmed_messages = trimmer.invoke(state["messages"])
    # Build prompt
    prompt = prompt_template.invoke({"messages": trimmed_messages})
    # Call the model
    response = model.invoke(prompt)
    # Return as a list of messages (the streaming pipeline expects an iterable)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)


# --- ADD MEMORY TO THE GRAPH ---
memory = MemorySaver()
langchain_app = workflow.compile(checkpointer=memory)  # We'll call this "langchain_app" to avoid overshadowing FastAPI's 'app'


# --- FASTAPI ENDPOINT ---
@app.post("/stream/")
async def stream_handler(request_data: QueryRequest):
    """
    Streams responses from the compiled StateGraph app (langchain_app).
    Terminates streaming after some given seconds.
    """
    query = request_data.query
    thread_id = request_data.thread_id

    # Construct config for memory saver usage (thread tracking, etc.)
    config = {"configurable": {"thread_id": thread_id}}

    # Build the input messages for the model
    input_messages = [HumanMessage(query)]
    
    start_time = time.time()  # Start timer
    response_content = ""

    # Stream from the compiled graph
    for chunk, metadata in langchain_app.stream(
        {"messages": input_messages},
        config,
        stream_mode="messages",
    ):
        if isinstance(chunk, AIMessage):
            response_content += chunk.content + " "

        # Stop if 20 seconds have passed
        if time.time() - start_time > 30:
            response_content += "\nStreaming ended after 30 seconds."
            break
    
    return {"response": response_content.strip()}


# --- LAUNCH SERVER ---
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

RuntimeError: asyncio.run() cannot be called from a running event loop